In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.weightstats import DescrStatsW

# Критерии Стьюдента
## Уровень кальция
Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.
### Ответ
Используем нормальное распределение - известна теоритическая дисперсия. Альтернатива двусторонняя.
$$H_0: \mu = \mu_0 = 9.5$$
$$H_1:\mu <\neq>  \mu_0$$
$$T = \frac{\bar{X}-\mu_0}{\frac{\sigma}{\sqrt{n}}}$$
$$p_{value}(T) \sim  N(0,1)$$

In [2]:
mu_0 = 9.5
sigma = 0.4
n = 160
X_hat = 9.57

In [3]:
T = (X_hat - mu_0) / (sigma / np.sqrt(n))
print ('Статистика равна', T)

Статистика равна 2.213594362117875


In [4]:
print('p_value равно', np.round(2 * (1 - sp.stats.norm.cdf(np.abs(T))), 4))

p_value равно 0.0269


Достигаемый уровень значимости меньше порога в 0.05 - значит нулевая гипотеза отклоняется в пользу альтернативы. Статистическая значимость гласит, средний уровень кальция отличается от 9.5. Предметную значимость точнее может сказать только специалист в предметной области.

## Shine bright like a diamond 
Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.model_selection.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

1. линейную регрессию с помощью LinearRegression без параметров
2. случайный лес с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости $\alpha=0.05$?

### Ответ
Выборки связные. Используем в качестве нулевого распределение критерий Стьюдента. Альтернатива двусторонняя.
$$H_0: \mu_1 = \mu_2$$
$$H_1:\mu_1 <\neq>  \mu_2$$
$$T = \frac{\bar{X_1}-\bar{X_2}}{\frac{S}{\sqrt{n}}}$$
$$p_{value}(T) \sim  St(\nu)$$

In [5]:
df = pd.read_csv('diamonds.txt', header = 0, sep = '\t')
df.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [6]:
X = df[['carat', 'depth', 'table', 'x', 'y', 'z']]
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
reg = LinearRegression().fit(X_train, y_train)
regr = RandomForestRegressor(random_state=1).fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [7]:
dif_reg = np.abs(y_test - reg.predict(X_test))
dif_regr = np.abs(y_test - regr.predict(X_test))

In [8]:
sp.stats.ttest_rel(dif_reg, dif_regr)

Ttest_relResult(statistic=13.01772978387856, pvalue=1.6551745751413995e-38)

Достигаемый уровень значимости меньше порога в 0.05 - значит нулевая гипотеза отклоняется. Статистическая значимость гласит, что эти два метода отличаются друг от друга.

In [9]:
DescrStatsW(dif_reg - dif_regr).tconfint_mean()

(74.28724532595444, 100.62452098634296)

Получается, случайный лес предсказывает цены бриллиантов в среднем на 70-100 долларов точнее.

## Результат
Был закреплен материал из лекций. 